In [1]:
# import Basic Dependeices
import numpy as np
import json
import random

In [2]:

# Importing necessary modules for building neural network using Keras
from tensorflow.keras.layers import Input, Dense
import tensorflow as tf

2024-04-07 01:36:03.229680: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:

# Usde to for Contextualisation and Other NLP Tasks.
import nltk
import ssl

# Set the SSL certificates path
ssl._create_default_https_context = ssl._create_unverified_context
nltk.download('punkt')

from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

[nltk_data] Downloading package punkt to /Users/abuzar/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
# Other
import json
import pickle
import warnings
warnings.filterwarnings("ignore")
print("Processing the Intents.....")

Processing the Intents.....


In [5]:
with open('./data/corpus.json') as json_data:
    intents = json.load(json_data)

In [6]:
words = []
classes = []
documents = []
ignore_words = ['?']
print("Looping through the Intents to Convert them to words, classes, documents and ignore_words.......")
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

Looping through the Intents to Convert them to words, classes, documents and ignore_words.......


In [7]:
print("Stemming, Lowering and Removing Duplicates.......")
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# remove duplicates
classes = sorted(list(set(classes)))

Stemming, Lowering and Removing Duplicates.......


In [8]:

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

360 documents
68 classes ['Customerservice', 'Products', 'account', 'account_management', 'add_gift_message', 'become_seller', 'bulk_discounts', 'cancelorder', 'change_or_cancel_item', 'change_shipping_address', 'community_and_social_interaction', 'compatibility_and_fitment', 'creator', 'customer_reviews', 'deals_and_discounts', 'detailssecure', 'discontinued_products', 'discount_code_not_working', 'email_newsletter', 'expedited_shipping', 'gift_wrapping', 'goodbye', 'greeting', 'guest', 'guest_orders', 'installation_services', 'international_shipping', 'leave_product_review', 'limited_edition_products', 'live_chat_support', 'lost_or_damaged_package', 'loyalty_program', 'multiple_promo_codes', 'name', 'number', 'order_as_gift', 'order_for_coming_soon_product', 'order_for_delivery_to_different_country', 'order_status_and_returns', 'out_of_stock_orders', 'payment', 'payment_options', 'phone_orders', 'price_adjustment_policy', 'price_matching_policy', 'product_demonstration', 'product_inf

In [9]:

print("Creating the Data for our Model.....")
training = []
output = []

Creating the Data for our Model.....


In [10]:

print("Creating an List (Empty) for Output.....")
output_empty = [0] * len(classes)

Creating an List (Empty) for Output.....


In [11]:
print("Creating Traning Set, Bag of Words for our Model....")
training_set = []
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training_set.append((bag, output_row))

Creating Traning Set, Bag of Words for our Model....


In [12]:
print("Shuffling Randomly......")
random.shuffle(training_set)

Shuffling Randomly......


In [13]:

print("Converting into Numpy Array for Faster Processing......")
bag_of_words, output_rows = zip(*training_set)
training = np.array(bag_of_words), np.array(output_rows)

Converting into Numpy Array for Faster Processing......


In [11]:

print("Creating Train and Test Lists.....")
train_x = training[0]
train_y = training[1]

Creating Train and Test Lists.....


In [12]:

print("Building Neural Network for Our Chatbot to be Contextual....")
print("Resetting graph data....")
tf.compat.v1.reset_default_graph()

Building Neural Network for Our Chatbot to be Contextual....
Resetting graph data....



In [13]:


net_input = Input(shape=(len(train_x[0]),))
net = Dense(8, activation=tf.nn.relu)(net_input)
net = Dense(8, activation=tf.nn.relu)(net)
net_output = Dense(len(train_y[0]), activation=tf.nn.softmax)(net)


In [14]:

# Creating the model
model = tf.keras.Model(inputs=net_input, outputs=net_output)

In [15]:

# Compiling the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [16]:

print("Training....")
# Assuming train_x and train_y are already defined
model.fit(train_x, train_y, epochs=1000, batch_size=8, verbose=1)

Training....
Epoch 1/1000

45/45 [==============================] - 3s 6ms/step - loss: 4.2032 - accuracy: 0.0194
Epoch 2/1000
45/45 [==============================] - 0s 7ms/step - loss: 4.1431 - accuracy: 0.0778
Epoch 3/1000
45/45 [==============================] - 0s 7ms/step - loss: 4.0132 - accuracy: 0.1222
Epoch 4/1000
45/45 [==============================] - 0s 7ms/step - loss: 3.7584 - accuracy: 0.1361
Epoch 5/1000
45/45 [==============================] - 0s 7ms/step - loss: 3.4728 - accuracy: 0.1333
Epoch 6/1000
45/45 [==============================] - 0s 7ms/step - loss: 3.2911 - accuracy: 0.1389
Epoch 7/1000
45/45 [==============================] - 0s 7ms/step - loss: 3.1612 - accuracy: 0.1611
Epoch 8/1000
45/45 [==============================] - 0s 9ms/step - loss: 3.0505 - accuracy: 0.1750
Epoch 9/1000
45/45 [==============================] - 0s 6ms/step - loss: 2.9493 - accuracy: 0.1861
Epoch 10/1000
45/45 [==============================] - 0s 7ms/step - loss: 2.8563 - ac

In [17]:

print("Saving the Model.......")
model.save('model.h5')

Saving the Model.......


In [18]:

print("Pickle is also Saved..........")
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

Pickle is also Saved..........


In [19]:

print("Loading Pickle.....")
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

Loading Pickle.....


In [20]:

with open('./data/corpus.json') as json_data:
    intents = json.load(json_data)

In [21]:

print("Loading the Model......")
# load our saved model
model = tf.keras.models.load_model('model.h5')

Loading the Model......


In [22]:

def clean_up_sentence(sentence):
    # It Tokenize or Break it into the constituents parts of Sentense.
    sentence_words = nltk.word_tokenize(sentence)
    # Stemming means to find the root of the word.
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

ERROR_THRESHOLD = 0.25
print("Chatbot is ready!")

Chatbot is ready!


In [23]:

def classify(sentence):
    # Prediction or To Get the Posibility or Probability from the Model
    results = model.predict(np.array([bow(sentence, words)]))[0]
    # Exclude those results which are Below Threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # Sorting is Done because heigher Confidence Answer comes first.
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1])) #Tuppl -> Intent and Probability
    return return_list

In [24]:

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # That Means if Classification is Done then Find the Matching Tag.
    if results:
        # Long Loop to get the Result.
        while results:
            for i in intents['intents']:
                # Tag Finding
                if i['tag'] == results[0][0]:
                    # Random Response from High Order Probabilities
                    return print(random.choice(i['responses']))

            results.pop(0)

In [25]:
def bow(sentence, words):
    # Tokenize the sentence
    sentence_words = nltk.word_tokenize(sentence)
    # Stemming the words
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    # Initialize the bag of words
    bag = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
    return np.array(bag)

In [27]:

while True:
    input_data = input("You- ")
    if input_data.lower() == 'quit':
        break
    else:
        answer = response(input_data)
        answer


1/1 [==============================] - 0s 204ms/step
Good to see you again!
1/1 [==============================] - 0s 102ms/step
You can ask me questions regarding Auto-Essential., and i will try to answer them
1/1 [==============================] - 0s 62ms/step
please use appropriate language
1/1 [==============================] - 0s 207ms/step
Shipping times vary depending on the destination and the shipping method chosen. Standard shipping usually takes 3-5 business days, while express shipping can take 1-2 business days.
